In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

In [3]:
preprocessing_fp = "preprocessing/combined_filtered/"

In [4]:
BATCH_SIZE = 32
EPOCHS = 500
IMG_SIZE = (224,224)
LABELS = ["female", "male"]

In [5]:
train_dataset = image_dataset_from_directory(os.path.join(preprocessing_fp, "train"),
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 29026 files belonging to 2 classes.


In [6]:
validation_dataset = image_dataset_from_directory(os.path.join(preprocessing_fp, "val"),
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 8275 files belonging to 2 classes.


In [7]:
# Initialize the Pretrained Model
feature_extractor = tf.keras.applications.resnet50.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer
input_ = tf.keras.Input(shape=(224, 224, 3))

# Set the feature extractor layer
x = feature_extractor(input_, training=False)

# Set the pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model object
model = tf.keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
model.fit(train_dataset, epochs=20, validation_data=validation_dataset)

Epoch 1/20
908/908 [==============================] - 66s 69ms/step - loss: 0.4682 - accuracy: 0.7684 - val_loss: 0.4028 - val_accuracy: 0.8141
Epoch 2/20
908/908 [==============================] - 61s 67ms/step - loss: 0.3757 - accuracy: 0.8277 - val_loss: 0.3680 - val_accuracy: 0.8331
Epoch 3/20
908/908 [==============================] - 61s 67ms/step - loss: 0.3534 - accuracy: 0.8437 - val_loss: 0.3608 - val_accuracy: 0.8389
Epoch 4/20
908/908 [==============================] - 61s 67ms/step - loss: 0.3394 - accuracy: 0.8517 - val_loss: 0.3568 - val_accuracy: 0.8399
Epoch 5/20
908/908 [==============================] - 61s 67ms/step - loss: 0.3320 - accuracy: 0.8529 - val_loss: 0.3519 - val_accuracy: 0.8418
Epoch 6/20
908/908 [==============================] - 61s 67ms/step - loss: 0.3253 - accuracy: 0.8560 - val_loss: 0.3512 - val_accuracy: 0.8427
Epoch 7/20
908/908 [==============================] - 62s 68ms/step - loss: 0.3175 - accuracy: 0.8633 - val_loss: 0.3546 - val_accuracy:

In [11]:
preprocessing_fp

'preprocessing/combined_filtered/'

In [27]:
mobilenet = tf.keras.models.load_model('model_tl_best_weights.h5')
resnet = tf.keras.models.load_model('model_tl_resnet_best_weights.h5')
densenet = tf.keras.models.load_model('model_tl_densenet_best_weights.h5')

In [28]:
models = [mobilenet, resnet, densenet]

In [41]:
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

def make_preds(model, gender=None):
    # FEMALE => 0
    # MALE => 1
    datasets = ["test_ori", "test_masked", "test_glasses", "test_make_up"]
    for data_name in datasets:
        y_true = []
        y_pred = []
        if gender is None:
            for i in os.listdir(preprocessing_fp + data_name + "/male"):
                img = Image.open(preprocessing_fp + data_name + "/male/" + i)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(1)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)

            for i in os.listdir(preprocessing_fp + data_name + "/female"):
                img = Image.open(preprocessing_fp + data_name + "/female/" + i)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(0)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)
        elif gender == 'male':
            for i in os.listdir(preprocessing_fp + data_name + "/male"):
                img = Image.open(preprocessing_fp + data_name + "/male/" + i)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(1)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)
        elif gender == 'female':
            for i in os.listdir(preprocessing_fp + data_name + "/female"):
                img = Image.open(preprocessing_fp + data_name + "/female/" + i)
                img = img.resize((224, 224))
                img = np.array(img)
                img = np.expand_dims(img, 0)

                y_true.append(0)
                y_pred.append(1 if model.predict(img) > 0.5 else 0)

        print(classification_report(y_true, y_pred))
        print()
        print(confusion_matrix(y_true, y_pred))

In [38]:
for model in models:
    make_preds(model)
    print("\n ================================================= \n")

              precision    recall  f1-score   support

           0       0.80      0.83      0.81      2144
           1       0.79      0.76      0.78      1860

    accuracy                           0.80      4004
   macro avg       0.80      0.80      0.80      4004
weighted avg       0.80      0.80      0.80      4004


[[1774  370]
 [ 438 1422]]
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      2144
           1       0.59      0.66      0.62      1862

    accuracy                           0.63      4006
   macro avg       0.63      0.63      0.63      4006
weighted avg       0.63      0.63      0.63      4006


[[1284  860]
 [ 636 1226]]
              precision    recall  f1-score   support

           0       0.78      0.61      0.69      2144
           1       0.64      0.80      0.71      1860

    accuracy                           0.70      4004
   macro avg       0.71      0.71      0.70      4004
weighted avg      

# Male only

In [42]:
for model in models:
    make_preds(model, 'male')
    print("\n ================================================= \n")

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.76      0.87      1860

    accuracy                           0.76      1860
   macro avg       0.50      0.38      0.43      1860
weighted avg       1.00      0.76      0.87      1860


[[   0    0]
 [ 438 1422]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.66      0.79      1862

    accuracy                           0.66      1862
   macro avg       0.50      0.33      0.40      1862
weighted avg       1.00      0.66      0.79      1862


[[   0    0]
 [ 636 1226]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

...

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.68      0.81      1862

    accuracy                           0.68      1862
   macro avg       0.50      0.34      0.41      1862
weighted avg       1.00      0.68      0.81      1862


[[   0    0]
 [ 591 1271]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.58      0.74      1860

    accuracy                           0.58      1860
   macro avg       0.50      0.29      0.37      1860
weighted avg       1.00      0.58      0.74      1860


[[   0    0]
 [ 774 1086]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.51      0.68      1860

    accuracy                           0.51      1860
   macro avg       0.50      0.25      0.34      1860
weighted avg       1.00      0.51      0.68      1860


[[  0   0]
 [912 948]]




/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

# Female only

In [43]:
for model in models:
    make_preds(model, 'female')
    print("\n ================================================= \n")

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.83      0.91      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.83      2144
   macro avg       0.50      0.41      0.45      2144
weighted avg       1.00      0.83      0.91      2144


[[1774  370]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.60      0.75      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.60      2144
   macro avg       0.50      0.30      0.37      2144
weighted avg       1.00      0.60      0.75      2144


[[1284  860]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

...

/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.61      0.76      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.61      2144
   macro avg       0.50      0.31      0.38      2144
weighted avg       1.00      0.61      0.76      2144


[[1312  832]
 [   0    0]]


/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       1.00      0.88      0.94      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.88      2144
   macro avg       0.50      0.44      0.47      2144
weighted avg       1.00      0.88      0.94      2144


[[1890  254]
 [   0    0]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      2144
           1       0.00      0.00      0.00         0

    accuracy                           0.95      2144
   macro avg       0.50      0.47      0.49      2144
weighted avg       1.00      0.95      0.97      2144


[[2034  110]
 [   0    0]]




/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/monash/.pyenv/versions/3.6.13/envs/fyp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le